<a href="https://colab.research.google.com/github/JiseonByeon96/PersonalProject/blob/main/%EC%83%9D%ED%99%9C%ED%8F%90%EA%B8%B0%EB%AC%BC%20%EB%89%B4%EC%8A%A4%EA%B8%B0%EC%82%AC%20%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import re
import json
from tqdm.notebook import tqdm
news_link = []


In [34]:
from tqdm.notebook import tqdm 

# page = 1, 2, 3, ~~~ 10
no_page = 10

news_link = []
for page in tqdm(range(1, no_page+1)): 
  url = 'https://search.naver.com/search.naver'

  query = {
      'where' : 'news',
      'query' : '생활폐기물',
      'start' : page   
  } 

  resp = requests.get(url, query)

  soup = BeautifulSoup(resp.content, 'lxml')
  #ul.type01 li dl dt a
  a_tag = soup.select("a[class=info]")  # a.info 로 지정 시 info가 포함된 모든 a tag; 즉 info press 와 info가 모두 파싱되므로 명확하게 이름 지정 

  for a in a_tag: # 10바퀴 * 10바퀴 = 100바퀴
    news_link.append(a['href'])
news_link

['https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=214&aid=0001097943',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0012184180',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=056&aid=0010984030',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=003&aid=0010329338',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=277&aid=0004841556',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=047&aid=0002301394',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0012184180',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0012184440',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=277&aid=0004841556',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=003&aid=0010329338',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=047&aid=0002301394',
 'https://

In [35]:
headers = {
    'Referer': 'https://hk-match.taboola.com/',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'
    } # 각자 본인의 User-Agent 정보 입력 

news = []
for link in tqdm(news_link):
    resp = requests.get(link, headers = headers)

    print(resp.url)
    soup = BeautifulSoup(resp.content, 'lxml')

    try: # 속보
        press = soup.select('div.press_logo img')[0]['title'] # 언론사 
        title = soup.select('div.article_info h3')[0].text # 타이틀

        publish_date = soup.select('div.sponsor span.t11')[0].text # 기사 입력 날짜 
        try: 
            modify_date = soup.select('div.sponsor span.t11')[1].text #최종 수정날짜 (만약 존재한다면)
        except:
            modify_date = ''  
        content = soup.select('div#articleBodyContents')[0].text.replace('// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}','').strip() # 앞 뒤의 공백 및 필요없는 문구 제거 
    except: # TV 연예홈 or 스포츠
        pass
        # if: # TV 연예홈

        # else: # 스포츠

    news.append([press, title, publish_date, modify_date, content ])

https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=214&aid=0001097943
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0012184180
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=056&aid=0010984030
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=003&aid=0010329338
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=277&aid=0004841556
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=047&aid=0002301394
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0012184180
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0012184440
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=277&aid=0004841556
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=003&aid=0010329338
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=047&aid=0002301394
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&

In [36]:
df_news = pd.DataFrame(news, columns=['언론사', '기사제목', '기사입력날짜', '최종수정날짜', '원문'])
df_news

,언론사,기사제목,기사입력날짜,최종수정날짜,원문
0,MBC,"환경부, 설 연휴 생활폐기물 특별대책 마련",2021.02.04. 오후 2:17,2021.02.04. 오후 2:30,환경부는 설 연휴를 앞두고 오는 8일부터 열흘 동안 생활폐기물 특별관리 대책을 추진...
1,연합뉴스,생활폐기물 직접 매립 금지된다…수도권은 2026년부터 시행,2021.02.04. 오후 12:00,,수도권 외 지역은 2030년부터…지자체별 폐기물 처리시설 확충 추진고성군 생활폐기물...
2,KBS,"수도권매립지, 설 연휴에 ‘13일 하루만 폐기물 반입’ 허용",2021.02.04. 오후 2:45,2021.02.04. 오후 2:51,수도권매립지관리공사는 오는 11일부터 14일까지 설 연휴 나흘 가운데 13일 하루만...
3,뉴시스,"경기도, 농촌 폐기물 불법 소각 방지 합동점검단 운영",2021.02.04. 오전 9:19,,"도, 31개 시·군 담당 공무원 156명으로 구성[수원=뉴시스] 경기도청 전경.(사..."
4,아시아경제,"경기도, 농촌 불법 폐기물 소각 '합동단속'",2021.02.04. 오전 7:09,,[아시아경제(수원)=이영규 기자] 경기도가 다음 달까지 농촌 지역 폐기물 불법 소각...
5,오마이뉴스,용인시 올해 잉여폐기물 1만3천톤 추가 발생…위탁업체에 맡겨 처리,2021.02.04. 오후 3:20,,[용인시민신문 임영조]폐기물량 '늘고' 처리는 '버겁고'···처리비용 크게 늘 듯 ...
6,연합뉴스,생활폐기물 직접 매립 금지된다…수도권은 2026년부터 시행,2021.02.04. 오후 12:00,,수도권 외 지역은 2030년부터…지자체별 폐기물 처리시설 확충 추진고성군 생활폐기물...
7,연합뉴스,"수도권매립지, 설 연휴 중 13일 하루 폐기물 정상 반입",2021.02.04. 오후 1:53,,수도권매립지 통합계량대[수도권매립지관리공사 제공. 재판매 및 DB 금지](인천=연합...
8,아시아경제,"경기도, 농촌 불법 폐기물 소각 '합동단속'",2021.02.04. 오전 7:09,,[아시아경제(수원)=이영규 기자] 경기도가 다음 달까지 농촌 지역 폐기물 불법 소각...
9,뉴시스,"경기도, 농촌 폐기물 불법 소각 방지 합동점검단 운영",2021.02.04. 오전 9:19,,"도, 31개 시·군 담당 공무원 156명으로 구성[수원=뉴시스] 경기도청 전경.(사..."
